# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = '../output_data/citylist.csv'
citylist = pd.read_csv(file)
citylist_clean = citylist.dropna()
citylist_clean.head()


,Lat,Lng,City Name,Country Code,Temp,Humidity,Clouds,Wind,Date
1,-54.80,-68.30,ushuaia,ar,46.40,49.0,75.0,4.70,1.601248e+09
2,64.89,-23.71,olafsvik,is,44.83,83.0,29.0,9.37,1.601248e+09
3,-22.97,-42.03,arraial do cabo,br,77.00,78.0,0.0,13.87,1.601247e+09
4,1.74,98.78,sibolga,id,76.24,71.0,99.0,2.89,1.601248e+09
5,25.93,81.98,bela,pk,78.19,71.0,0.0,1.88,1.601248e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = citylist_clean[['Lat', 'Lng']]

humidity = citylist_clean['Humidity']

gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, point_radius=20)
                                


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#narrow down the dataframe to ideal conditions
ideal = citylist_clean.loc[(citylist_clean['Temp']<80) & (citylist_clean['Temp']>70) & (citylist_clean['Wind']<10) & (citylist_clean['Clouds']==0.0)]
ideal.head()

,Lat,Lng,City Name,Country Code,Temp,Humidity,Clouds,Wind,Date
5,25.93,81.98,bela,pk,78.19,71.0,0.0,1.88,1.601248e+09
57,-17.71,-39.25,caravelas,br,73.24,81.0,0.0,7.92,1.601248e+09
80,-27.80,-64.26,santiago del estero,ar,70.29,25.0,0.0,8.93,1.601248e+09
163,30.48,-115.95,san quintin,mx,73.49,65.0,0.0,7.76,1.601248e+09
351,34.93,43.49,bayji,iq,78.24,28.0,0.0,7.34,1.601248e+09


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = ideal
len(hotel_df)

6

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",  
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
hotel_df['Hotel Name']=""
for x in range(len(hotel_df)):
    
    lat=hotel_df.iloc[x,0] 
    lng=hotel_df.iloc[x,1]
    params['location']=f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for {hotel_df.iloc[x, 2]}.")
    response = requests.get(base_url, params=params).json()
    print(params['location'])
    # extract results
    
    
    try:
        results = response['results'][0]['name']
        print(results)
        hotel_df.iloc[x, 9] = results
                
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------") 

C:\Users\mennbou\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Retrieving Results for bela.
25.93,81.98
Jameel Khan & Services
------------
Retrieving Results for caravelas.


C:\Users\mennbou\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


-17.71,-39.25
Mateus' S/A
------------
Retrieving Results for santiago del estero.
-27.8,-64.26
Hotel Nuevo America
------------
Retrieving Results for san quintin.
30.48,-115.95
Refugio para mascotas Good Boy A. C.
------------
Retrieving Results for bayji.
34.93,43.49
مدرسة بيجي الابتدائية للبنات
------------
Retrieving Results for gat.
31.61,34.76
שדרות גת 88 קרית גת
------------


In [9]:
hotel_df.head(6)

,Lat,Lng,City Name,Country Code,Temp,Humidity,Clouds,Wind,Date,Hotel Name
5,25.93,81.98,bela,pk,78.19,71.0,0.0,1.88,1.601248e+09,Jameel Khan & Services
57,-17.71,-39.25,caravelas,br,73.24,81.0,0.0,7.92,1.601248e+09,Mateus' S/A
80,-27.80,-64.26,santiago del estero,ar,70.29,25.0,0.0,8.93,1.601248e+09,Hotel Nuevo America
163,30.48,-115.95,san quintin,mx,73.49,65.0,0.0,7.76,1.601248e+09,Refugio para mascotas Good Boy A. C.
351,34.93,43.49,bayji,iq,78.24,28.0,0.0,7.34,1.601248e+09,مدرسة بيجي الابتدائية للبنات
396,31.61,34.76,gat,ly,79.00,77.0,0.0,8.30,1.601248e+09,שדרות גת 88 קרית גת


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_2 = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
locations = citylist_clean[['Lat', 'Lng']]

humidity = citylist_clean['Humidity']

gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, point_radius=20)

markers = gmaps.marker_layer(locations_2,
    info_box_content=[f"Hotel Name: {x}" for x in hotel_info])
fig.add_layer(markers)                               


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))